In [ ]:
import pandas as pd
import csv
from itertools import combinations
import re



file = pd.read_csv('sampleInputTable.csv')
print(file)
print('\n')

with open('fds.txt', 'r') as f:
    rows = [row.strip() for row in f]

functionaldependencies = {}
for row in rows:
    determ, depend = row.split(" -> ")
    determ = determ.split(", ")
    functionaldependencies[tuple(determ)] = depend.split(", ")
print('Functional Dependencies')
print(functionaldependencies)


maximum_normal_form = input("\nSelect the highest normal form to normalize from the below list \n1 for 1NF\n2 for 2NF\n3 for 3NF\nB for BCNF\n4 for 4NF\n5 for 5NF: ")
if maximum_normal_form in ["1", "2", "3", "4", "5"]:
    maximum_normal_form = int(maximum_normal_form)

find_highest_normal_form = int(
    input('\nDetermine the highest normal form for the given relation? (1: Yes, 2: No): '))
high_normalform = 'not normalized'

pk = input("\nEnter the primary key seperated with commas : ")
primary_key = pk.split(', ')
print('\n')

keys = ()
keys = tuple(key for key in primary_key)

primary_key = keys


def check_comma(series):
    return any(',' in item for item in series)



def input_parser(file):
    file = file.astype(str)
    columns_with_commas = [
        col for col in file.columns if check_comma(file[col])]

    for col in columns_with_commas:
        file[col] = file[col].str.split(
            ',').apply(lambda x: [item.strip() for item in x])

    return file
file = input_parser(file)

multi_value = {}
if not maximum_normal_form == 'B' and maximum_normal_form >= 4:
    with open('mvd.txt', 'r') as files:
        multi_value_dependency_lines = [line.strip() for line in files]

    print(multi_value_dependency_lines)

    for mvd in multi_value_dependency_lines:
        determ, depend = mvd.split(" ->> ")
        determ = determ.split(
            ", ") if ", " in determ else [determ]
        determ_str = str(determ)
        if determ_str in multi_value:
            multi_value[determ_str].append(depend)
        else:
            multi_value[determ_str] = [depend]


def check_is_superkey(rel, determ):
    grouped = rel.groupby(
        list(determ)).size().reset_index(name='count')
    return not any(grouped['count'] > 1)

def check_list_or_set(item):
    return isinstance(item, (list, set))

"""def powerset(s):
    x = len(s)
    for i in range(1 << x):
        yield [s[j] for j in range(x) if (i & (1 << j)) > 0]"""


def bcnf_decomp(rel, functionaldependencies):
    decomposed_rels = [rel]
    for determ, depends in functionaldependencies.items():
        if set(determ).issubset(rel.columns) and not check_is_superkey(rel, determ):
            depend_cols = list(determ) + depends
            new_rel1 = rel[depend_cols].drop_duplicates()
            remaining_cols = list(set(rel.columns) - set(depends))
            new_rel2 = rel[remaining_cols].drop_duplicates()
            decomposed_rels = [new_rel for new_rel in decomposed_rels if new_rel is not rel]
            decomposed_rels.extend([new_rel1, new_rel2])
    return decomposed_rels



def is_in_1nf(rel):
    if rel.empty:
        return False

    for column in rel.columns:
        if any(isinstance(value, (list, dict, set)) for value in rel[column]):
            return False

    return True



def is_in_2nf(primary_key, functionaldependencies):
    for determs, depend in functionaldependencies.items():
        superkey = True
        for x in determs:
            if x not in file.columns:
                superkey = False
                break
        if superkey:
            for y in depend:
                if y not in file.columns or y not in determs:
                    return False
    return True



def is_in_3nf(rels, functionaldependencies):
    all_attributes = set()
    for rel in rels:
        all_attributes.update(rels[rel].columns)

    for determ, depends in functionaldependencies.items():
        if all(attr not in all_attributes for attr in determ):
            continue
        if any(attr in all_attributes for attr in depends):
            continue
        return False

    return True



def is_in_bcnf(rels, primary_key, functionaldependencies):
    for rel in rels:
        for determ, depends in functionaldependencies.items():
            if set(determ).issubset(rel.columns):
                if not check_is_superkey(rel, determ):
                    return False
    return True


def is_in_4nf(rels, multi_value):
    for rel in rels:
        for determ, depends in multi_value.items():
            if isinstance(determ, tuple):
                determ_cols = list(determ)
            else:
                determ_cols = [determ]

            if all(col in rel.columns for col in determ_cols + depends):
                grouped = rel.groupby(determ_cols)[
                    depends].apply(set).reset_index()
                if len(grouped) < len(rel):
                    print(f"violation of multi-valued functional dependencies: {determ} ->> {depends}")
                    return False
    return True



def is_in_5nf(rels):
    candidate_keys_dict = {}

    for i, rel in enumerate(rels):
        print(rel)
        user_input = input("Enter the candidate keys : ")
        print('\n')
        tuples = re.findall(r'\((.*?)\)', user_input)
        candidate_keys = [tuple(map(str.strip, t.split(','))) for t in tuples]
        candidate_keys_dict[i] = candidate_keys

    print(f"The Candidate Keys for the given relations: {candidate_keys_dict}\n")

    for j, rel in enumerate(rels):
        candidate_keys = candidate_keys_dict[j]
        data_tuples = [tuple(row) for row in rel.to_numpy()]

        def project(data, attributes):
            return {tuple(row[attr] for attr in attributes) for row in data}

        def is_superkey(attributes):
            return any(set(key).issubset(attributes) for key in candidate_keys)

        for i in range(1, len(rel.columns)):
            for attrs in combinations(rel.columns, i):
                if is_superkey(attrs):
                    continue

                projected_data = project(data_tuples, attrs)
                complement_attrs = set(rel.columns) - set(attrs)
                complement_data = project(data_tuples, complement_attrs)

                joined_data = {(row1 + row2) for row1 in projected_data for row2 in complement_data}
                if set(data_tuples) != joined_data:
                    print(f"Does not satisfy 5NF, check attributes: {attrs}" )
                    return False, candidate_keys_dict

    return True, candidate_keys_dict




def decomposing_to_5nf(dataframe, candidate_keys):

    def project(df, attributes):
        return df[list(attributes)].drop_duplicates().reset_index(drop=True)

    def is_lossless(df, df1, df2):
        common_columns = set(df1.columns) & set(df2.columns)
        if not common_columns:
            return False
        joined_df = pd.merge(df1, df2, how='inner', on=list(common_columns))
        return df.equals(joined_df)

    decomposed_tables = [dataframe]

    for key in candidate_keys:
        new_tables = []
        for table in decomposed_tables:
            if set(key).issubset(set(table.columns)):
                table1 = project(table, key)
                remaining_columns = set(table.columns) - set(key)
                table2 = project(table, remaining_columns | set(key))

                if is_lossless(table, table1, table2):
                    new_tables.extend([table1, table2])
                else:
                    new_tables.append(table)
            else:
                new_tables.append(table)
        decomposed_tables = new_tables

    return decomposed_tables


def fifth_normalization_form(rels, primary_key, functionaldependencies):
    five_rels = []
    five_flag, candidate_keys_dict = is_in_5nf(rels)

    if five_flag:
        return rels, five_flag
    else:
        i = 0
        for rel in rels:
            candidate_keys = candidate_keys_dict[i]
            i += 1
            decomposed_rels = decomposing_to_5nf(rel, candidate_keys)
            five_rels.append(decomposed_rels)

    return five_rels, five_flag




def fourth_normalization_form(rels, multi_value):
    four_rels = []
    four_flag = is_in_4nf(rels, multi_value)

    if four_flag:
        return rels, four_flag
    else:
        for rel in rels:
            for determ, depends in multi_value.items():
                for depend in depends:
                    if isinstance(determ, tuple):
                        determ_cols = list(determ)
                    else:
                        determ_cols = [determ]

                    if all(col in rel.columns for col in determ_cols + [depend]):
                        grouped = rel.groupby(determ_cols)[
                            depend].apply(set).reset_index()
                        if len(grouped) < len(rel):
                            table_1 = rel[determ_cols +
                                               [depend]].drop_duplicates()
                            table_2 = rel[determ_cols + [col for col in rel.columns if col not in [
                                depend] + determ_cols]].drop_duplicates()


                            four_rels.extend([table_1, table_2])

                            break
                else:
                    continue
                break
            else:
                four_rels.append(rel)

    if len(four_rels) == len(rels):
        return four_rels
    else:
        return fourth_normalization_form(four_rels, multi_value)






def third_normalization_form(rels, primary_key, functionaldependencies):
    three_rels = {}
    original_rel = rels
    three_flag = is_in_3nf(rels, functionaldependencies)

    if three_flag:
        return rels, three_flag
    else:
        depend_keys = list(functionaldependencies.keys())
        for rel in rels:
            original_rel = rels[rel]
            for determ, depends in functionaldependencies.items():
                modified_depends = [
                    dep + '_fk' if (dep,) in depend_keys else dep for dep in depends]

                cols = list(determ) + depends
                three_rels[tuple(determ)] = rels[rel][cols].drop_duplicates(
                ).reset_index(drop=True)

                rename_dict = {dep: modified_dep for dep,
                               modified_dep in zip(depends, modified_depends)}
                three_rels[tuple(determ)].rename(
                    columns=rename_dict, inplace=True)

        junc_cols = []

        rel_name = ''
        for rel in three_rels:
            rel_name += "_".join(rel)
            junc_cols.append(rel)

        if len(junc_cols) > 1:
            jun_cols = list(junc_cols)
            cols = [element for tup in jun_cols for element in tup]
            temp_df = original_rel[cols].drop_duplicates(
            ).reset_index(drop=True)

            renamed_cols = [col + '_fk' for col in cols]
            temp_df.columns = renamed_cols + \
                [col for col in temp_df.columns if col not in cols]

            temp_df[rel_name] = range(1, len(temp_df) + 1)
            col_order = [rel_name] + renamed_cols
            temp_df = temp_df[col_order]
            three_rels[rel_name] = temp_df

        return three_rels, three_flag





def bc_normal_form(rels, primary_key, functionaldependencies):
    rels = list(rels.values())
    bcnf_rels = []
    bcnf_flag = is_in_bcnf(rels, primary_key, functionaldependencies)

    if bcnf_flag:
        return rels, bcnf_flag
    else:
        for rel in rels:
            bcnf_decomp_rel = bcnf_decomp(
                rel, functionaldependencies)
            if len(bcnf_decomp_rel) == 1:
                bcnf_rels.append(bcnf_decomp_rel)
            else:
                rels.extend(bcnf_decomp_rel)

    return bcnf_rels, bcnf_flag



def second_normalization_form(rel, primary_key, functionaldependencies):
    relations = {}
    original_rel = rel
    two_flag = is_in_2nf(primary_key, functionaldependencies)

    if two_flag:
        relations[primary_key] = rel
        return relations, two_flag
    else:
        depend_keys = list(functionaldependencies.keys())
        for determ, depends in functionaldependencies.items():
            modified_depends = [
                dep + '_fk' if (dep,) in depend_keys else dep for dep in depends]

            cols = list(determ) + depends
            relations[tuple(determ)] = rel[cols].drop_duplicates(
            ).reset_index(drop=True)

            rename_dict = {dep: modified_dep for dep,
                           modified_dep in zip(depends, modified_depends)}
            relations[tuple(determ)].rename(
                columns=rename_dict, inplace=True)

        junc_cols = []
        rel_name = ''
        for rel in relations:
            if set(rel).issubset(primary_key):
                rel_name += "_".join(rel)
                junc_cols.append(rel)

        if len(junc_cols) > 1:
            jun_cols = list(junc_cols)
            cols = [element for tup in jun_cols for element in tup]
            temp_df = original_rel[cols].drop_duplicates(
            ).reset_index(drop=True)

            renamed_cols = [col + '_fk' for col in cols]
            temp_df.columns = renamed_cols + \
                [col for col in temp_df.columns if col not in cols]

            temp_df[rel_name] = range(1, len(temp_df) + 1)
            col_order = [rel_name] + renamed_cols
            temp_df = temp_df[col_order]
            relations[rel_name] = temp_df


        return relations, two_flag



def first_normalization_form(rel):
    if is_in_1nf(rel):
        return rel, True

    for col in rel.columns:
        if rel[col].apply(check_list_or_set).any():
            rel = rel.explode(col)

    return rel, is_in_1nf(rel)


def output(dtype):
    if "int" in str(dtype):
        return "INT"
    elif "float" in str(dtype):
        return "FLOAT"
    elif "datetime" in str(dtype):
        return "DATETIME"
    elif "object" in str(dtype):
        return "VARCHAR(255)"
    else:
        return "TEXT"


def sql_query_generator_for_1NF(primary_keys, x):
    table_name = "_".join(primary_keys) + "_table"

    query = f"CREATE TABLE {table_name} (\n"

    for col, dtype in zip(x.columns, x.dtypes):
        if col in primary_keys:
            query += f"  {col} {output(dtype)} PRIMARY KEY,\n"
        else:
            query += f"  {col} {output(dtype)},\n"


    query = query.rstrip(',\n') + "\n);"

    print(query)



def sql_query_generator_for_2_3(relations):
    for rel in relations:
        primary_keys = rel
        primary_keys = (primary_keys,) if isinstance(primary_keys, str) else primary_keys
        table_name = "_".join(primary_keys) + '_table'
        rel = relations[rel]

        query = f"CREATE TABLE {table_name} (\n"

        for col, dtype in zip(rel.columns, rel.dtypes):
            if col in primary_keys:
                query += f"  {col} {output(dtype)} PRIMARY KEY,\n"
            elif '_fk' in col:
                query += f" FOREIGN KEY ({col}) REFERENCES {col.replace('_fk','')}_table({col.replace('_fk','')}),\n"
            else:
                query += f"  {col} {output(dtype)},\n"


        query = query.rstrip(',\n') + "\n);"

        print(query)


def sql_query_generator_for_BCNF_4_5(relations):
    for rel in relations:
        primary_key = rel.columns[0]
        table_name = f'{primary_key}_table'

        query = f"CREATE TABLE {table_name} (\n"

        for col, dtype in zip(rel.columns, rel.dtypes):
            if col == primary_key:
                query += f"  {col} {output(dtype)} PRIMARY KEY,\n"
            elif '_fk' in col:
                query += f" FOREIGN KEY ({col}),\n"
            else:
                query += f"  {col} {output(dtype)},\n"
        query = query.rstrip(',\n') + "\n);"

        print(query)

if maximum_normal_form >= 1 or maximum_normal_form == 'B':
    onenf_table, one_flag = first_normalization_form(file)

    if one_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 1NF'

    if maximum_normal_form == 1 and one_flag:
        print('The table is already in 1NF\n')
        sql_query_generator_for_1NF(primary_key, onenf_table)


if maximum_normal_form >= 2 or maximum_normal_form == 'B':
    twonf_tables, two_flag = second_normalization_form(onenf_table, primary_key, functionaldependencies)

    if maximum_normal_form == 2 and two_flag and one_flag:
        print('The table is already in 2NF\n')
        sql_query_generator_for_2_3(twonf_tables)
    elif one_flag and two_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 2NF'


if maximum_normal_form >= 3 or maximum_normal_form == 'B':
    threenf_tables, three_flag = third_normalization_form(twonf_tables, primary_key, functionaldependencies)

    if one_flag and two_flag and three_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 3NF'

    if maximum_normal_form == 3 and three_flag and two_flag and one_flag:
        print('The table is already in 3NF\n')

    sql_query_generator_for_2_3(threenf_tables)

if maximum_normal_form >= 4 or maximum_normal_form == 'B':
    bcnf_tables, bcnf_flag = bc_normal_form(threenf_tables, primary_key, functionaldependencies)

    if one_flag and two_flag and three_flag and bcnf_flag:
        high_normalform = 'The Highest Normal Form of the given table is: BCNF'

    if maximum_normal_form == 'B'and bcnf_flag and three_flag and two_flag and one_flag:
        print('The table is already in BCNF\n')

    sql_query_generator_for_BCNF_4_5(bcnf_tables)

if not maximum_normal_form == 'B' and maximum_normal_form >= 4:
    fournf_tables, four_flag = fourth_normalization_form(bcnf_tables, multi_value)

    if one_flag and two_flag and three_flag and bcnf_flag and four_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 4NF'

    if maximum_normal_form == 4 and four_flag and bcnf_flag and three_flag and two_flag and one_flag:
        print('The table is already in 4NF')
        print('\n')

    sql_query_generator_for_BCNF_4_5(fournf_tables)

if not maximum_normal_form == 'B' and maximum_normal_form >= 5:
    fivenf_tables, five_flag = fifth_normalization_form(fournf_tables, primary_key, functionaldependencies)

    if one_flag and two_flag and three_flag and bcnf_flag and four_flag and five_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 5NF'

    if maximum_normal_form == 5 and five_flag and four_flag and bcnf_flag and three_flag and two_flag and one_flag:
        print('The table is already in 5NF\n')

    sql_query_generator_for_BCNF_4_5(fivenf_tables)

if find_highest_normal_form == 1:
    print(f"\n{high_normalform}\n")


   StudentID FirstName  LastName   Course  Professor  ProfessorEmail  \
0        101      John       Doe  Math101   Dr.Smith   smith@mst.edu   
1        102      Jane       Roe  Math101   Dr.Smith   smith@mst.edu   
2        103   Arindam    Khanda    CS101   Dr.Jones   jones@mst.edu   
3        104      Jose  Franklin   Bio101  Dr.Watson  watson@mst.edu   
4        105       Ada  Lovelace    CS101   Dr.Jones   jones@mst.edu   

  CourseStart CourseEnd  
0      1/1/23   5/30/23  
1      1/1/23   5/30/23  
2      2/1/23   6/15/23  
3      3/1/23   7/20/23  
4      2/1/23   6/15/23  


Functional Dependencies
{('StudentID',): ['FirstName', 'LastName'], ('Course',): ['CourseStart', 'CourseEnd', 'Professor'], ('Professor',): ['ProfessorEmail']}

Select the highest normal form to normalize from the below list 
1 for 1NF
2 for 2NF
3 for 3NF
B for BCNF
4 for 4NF
5 for 5NF: 1

Determine the highest normal form for the given relation? (1: Yes, 2: No): 1

Enter the primary key seperated with com

# New Section